In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import re
import nltk
import spacy
import pandas as pd 
import numpy as np
from statistics import variance
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/Dataset/instances.csv")

In [ ]:
dataset.head()

,id,postTimestamp,postText,postMedia,targetTitle,targetDescription,targetKeywords,targetParagraphs,targetCaptions
0,858462320779026433,Sat Apr 29 23:25:41 +0000 2017,['UK’s response to modern slavery leaving vict...,[],‘Inexcusable’ failures in UK’s response to mod...,“Inexcusable” failures in the UK’s system for ...,"modern slavery, Department For Work And Pensio...",['Thousands of modern slavery victims have\xa0...,['modern-slavery-rex.jpg']
1,858421020331560960,Sat Apr 29 20:41:34 +0000 2017,['this is good'],[],Donald Trump Appoints Pro-Life Advocate as Ass...,President Donald Trump has appointed pro-life ...,"Americans United for Life, Dr. Charmaine Yoest...",['President Donald Trump has appointed the\xa0...,"['In this July 1, 2010 file photo, Dr. Charmai..."
2,858368123753435136,Sat Apr 29 17:11:23 +0000 2017,"['The ""forgotten"" Trump roast: Relive his brut...",[],The ‘forgotten’ Trump roast: Relive his brutal...,President Trump won't be at this year's White ...,"trump whcd, whcd, white house correspondents d...",['When the\xa0White House correspondents’ dinn...,"[""President Trump will not attend this year's ..."
3,858323428260139008,Sat Apr 29 14:13:46 +0000 2017,['Meet the happiest #dog in the world!'],[],"Meet The Happiest Dog In The World, Maru The H...","The article is about Maru, a husky dog who has...","Maru, husky, dogs, pandas, furball, instagram",['Adorable is probably an understatement. This...,"['Maru ', 'Maru', 'Maru', 'Maru', 'Maru']"
4,858283602626347008,Sat Apr 29 11:35:31 +0000 2017,"[""Tokyo's subway is shut down amid fears over ...",[],Tokyo's subway is shut down amid fears over an...,"The temporary suspension, which lasted ten min...","Tokyo,subway,shut,fears,North,Korean,attack","[""One of Tokyo's major subways systems says it...","[""All nine lines of Tokyo's subway system were..."


In [ ]:
dataset.shape

(19538, 9)

In [ ]:
postText = dataset["postText"]

In [ ]:
type(postText)

pandas.core.series.Series

In [ ]:
postText

0        ['UK’s response to modern slavery leaving vict...
1                                         ['this is good']
2        ['The "forgotten" Trump roast: Relive his brut...
3                 ['Meet the happiest #dog in the world!']
4        ["Tokyo's subway is shut down amid fears over ...
                               ...                        
19533    ["Brazil soccer team and pilot's final intervi...
19534                                   ['😱😱😱😱😱😱😱😱😱😱😱😱😱😱']
19535    ['Frenchs Forest high school may have to make ...
19536                                   ['Oh Jeff… #bruh']
19537    ['Richard Sherman weighs in on Cam Newton’s st...
Name: postText, Length: 19538, dtype: object

In [ ]:
def preProcess(s):
  l = []
  for item in s:
    if(type(item)==str):
      item = item.replace('["','')
      item = item.replace('"]','')
      item = item.replace("']",'')
      item = item.replace("['",'')
    l.append(item)
  return l

In [ ]:
dataset["postText"] = pd.DataFrame(preProcess(dataset["postText"]))

In [ ]:
postText = dataset["postText"]



In [ ]:
postText

0        UK’s response to modern slavery leaving victim...
1                                             this is good
2        The "forgotten" Trump roast: Relive his brutal...
3                     Meet the happiest #dog in the world!
4        Tokyo's subway is shut down amid fears over an...
                               ...                        
19533    Brazil soccer team and pilot's final interview...
19534                                       😱😱😱😱😱😱😱😱😱😱😱😱😱😱
19535    Frenchs Forest high school may have to make wa...
19536                                       Oh Jeff… #bruh
19537    Richard Sherman weighs in on Cam Newton’s stru...
Name: postText, Length: 19538, dtype: object

In [ ]:
id = dataset["id"]

In [ ]:
dataset.isnull().sum()

id                      0
postTimestamp           0
postText                0
postMedia               0
targetTitle             0
targetDescription     943
targetKeywords       7229
targetParagraphs        0
targetCaptions          0
dtype: int64

In [ ]:
postText[261]

''

In [ ]:
postText[195]

"There are echoes of the US election in France's presidential race, writes @DavidAndelman for @CNNopinion"

In [ ]:
type(postText)

pandas.core.series.Series

In [ ]:
type(postText[0])

str

In [ ]:
postText[0]

'UK’s response to modern slavery leaving victims destitute while abusers go free'

In [ ]:
print(postText)

0        UK’s response to modern slavery leaving victim...
1                                             this is good
2        The "forgotten" Trump roast: Relive his brutal...
3                     Meet the happiest #dog in the world!
4        Tokyo's subway is shut down amid fears over an...
                               ...                        
19533    Brazil soccer team and pilot's final interview...
19534                                       😱😱😱😱😱😱😱😱😱😱😱😱😱😱
19535    Frenchs Forest high school may have to make wa...
19536                                       Oh Jeff… #bruh
19537    Richard Sherman weighs in on Cam Newton’s stru...
Name: postText, Length: 19538, dtype: object


Convert emoji into texts 

In [ ]:
!pip install demoji


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 KB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=49e80f46fcfe27d49f90b8541ac718cb0ff192c0ce29988184cb88891372d133
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [ ]:
# Import required libraries
import emoji

# Text from where you want to replace emojis
text = postText[19534]

replaced_text = emoji.demojize(text, delimiters=(" ", " "))

# Printing replaced text
print(replaced_text)

 face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear  face_screaming_in_fear 


In [ ]:
import demoji
demoji.download_codes()

<ipython-input-28-eb011a9810ad>:2: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:

# Text from where you want to convert emojis
def replaceEmojiToText(postText):
  l=[]
  for text in postText:
    emojis = demoji.findall(text)
    if(len(emojis.values()) == 0):
      l.append("")
    else:
      l.append(list(emojis.values()))
  return l



In [ ]:
emojiList = replaceEmojiToText(postText)

In [ ]:
print(emojiList)

['', '', '', '', '', '', '', '', '', '', '', ['eyes'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['movie camera'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ['folded hands'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '

In [ ]:
postText[11]

'Players to know, QBs to watch and more 👀\\n\\nGuide to Round 2 of the #NFLDraft'

In [ ]:
postText[0]

'UK’s response to modern slavery leaving victims destitute while abusers go free'

In [ ]:
#remove stopwords from postText , removal punctuation, lower case
def removeStopWords(postText):
  l = []
  for item in postText:
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(item)
    #print(word_tokens)
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    new_postText = " ".join(filtered_sentence)
    pText = re.sub(r'[^\w\s]', '', new_postText)
    #print(new_postText)
    l.append(pText)
  return l
    

In [ ]:
postText[0]

'UK’s response to modern slavery leaving victims destitute while abusers go free'

In [ ]:
postText[19534]

'😱😱😱😱😱😱😱😱😱😱😱😱😱😱'

In [ ]:
pText = removeStopWords(postText)

In [ ]:
pText[19534]

''

In [ ]:
pText[261]

''

In [ ]:
pText[0]

'uk  response modern slavery leaving victims destitute abusers go free'

In [ ]:
pText[11]

'players know  qbs watch nnguide round 2  nfldraft'

In [ ]:
sentence = ['this','is','a','sentence']
s=(" ".join(sentence))
print(s)

this is a sentence


Placed texts equivalent to emoji

In [ ]:
def concatEmojiText(postText, emojiList):
  text = []
  for i in range(len(postText)):
    if(len(emojiList[i])==0):
      text.append(postText[i])
      continue
  
    joined = ' '.join(emojiList[i])
    #print(joined)
    if(len(postText[i]) == 0):
      text.append(joined)
    else:
      l=postText[i] + " " + joined
      text.append(l)
  return text

    
  



    
 
   

In [ ]:
text = concatEmojiText(pText, emojiList)

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Lemmatization(split into root word) of post texts 

In [ ]:
def lemmatizePostText(pText):
  l = []
  lemmatizer = WordNetLemmatizer()
  for item in pText:
    word_list = nltk.word_tokenize(item)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    l.append(lemmatized_output)
  return l




In [ ]:
postTxt = lemmatizePostText(text)

In [ ]:
# Remove stop words using inbuilt python module 

In [ ]:
postT=removeStopWords(postTxt)

In [ ]:
print(postT[0])

uk response modern slavery leaving victim destitute abuser go free


In [ ]:
data = pd.DataFrame()
data["id"] = id
data["postText"] = postT


In [ ]:
data.head()

,id,postText
0,858462320779026433,uk response modern slavery leaving victim dest...
1,858421020331560960,good
2,858368123753435136,forgotten trump roast relive brutal 2004 thras...
3,858323428260139008,meet happiest dog world
4,858283602626347008,tokyo subway shut amid fear imminent north kor...


In [ ]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [ ]:
from nltk.corpus import sentiwordnet as swn
import ssl
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

In [ ]:
pos=neg=obj=count=0
lemmatizer = WordNetLemmatizer()

postagging = []

for review in data["postText"]:
    list = word_tokenize(review)
    postagging.append(nltk.pos_tag(list))

data['pos_tags'] = postagging

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
        return []

    #Lemmatization
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0
    
    ###################################################################################
senti_score = []
senti_score_absolute = []

for pos_val in data['pos_tags']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg) #actual value of sentiment score aggregate 
    senti_score_absolute.append(pos + neg) #absolute value of sentiment score aggregate 
    pos=neg=0    
    
data['senti_score'] = senti_score #Actual sum of sentiment score of words of post text
data['senti_score_absolute'] = senti_score_absolute  #absolute sum of sentiment score of words of post text
print(data['senti_score'])
print(data['senti_score_absolute'])

print(data.head)


0       -0.125
1        0.750
2        0.000
3        0.875
4        0.000
         ...  
19533   -0.500
19534   -0.875
19535   -0.125
19536    0.000
19537    0.000
Name: senti_score, Length: 19538, dtype: float64
0        0.875
1        0.750
2        0.750
3        0.875
4        0.000
         ...  
19533    0.500
19534    0.875
19535    0.375
19536    0.000
19537    0.000
Name: senti_score_absolute, Length: 19538, dtype: float64
<bound method NDFrame.head of                        id                                           postText  \
0      858462320779026433  uk response modern slavery leaving victim dest...   
1      858421020331560960                                               good   
2      858368123753435136  forgotten trump roast relive brutal 2004 thras...   
3      858323428260139008                            meet happiest dog world   
4      858283602626347008  tokyo subway shut amid fear imminent north kor...   
...                   ...                            

In [ ]:
data.head()

,id,postText,pos_tags,senti_score,senti_score_absolute
0,858462320779026433,uk response modern slavery leaving victim dest...,"[(uk, JJ), (response, NN), (modern, JJ), (slav...",0.125,0.625
1,858421020331560960,good,"[(good, JJ)]",0.750,0.750
2,858368123753435136,forgotten trump roast relive brutal 2004 thras...,"[(forgotten, JJ), (trump, NN), (roast, NN), (r...",0.000,0.750
3,858323428260139008,meet happiest dog world,"[(meet, NN), (happiest, JJS), (dog, NN), (worl...",0.875,0.875
4,858283602626347008,tokyo subway shut amid fear imminent north kor...,"[(tokyo, JJ), (subway, NN), (shut, NN), (amid,...",0.000,0.000


In [ ]:
data.tail()

,id,postText,pos_tags,senti_score,senti_score_absolute
19533,804250183642976256,brazil soccer team pilot final interview plane...,"[(brazil, NN), (soccer, NN), (team, NN), (pilo...",-0.500,0.500
19534,804156272086020096,face screaming fear,"[(face, NN), (screaming, VBG), (fear, NN)]",0.000,0.000
19535,804149798651588608,french forest high school may make way highris...,"[(french, JJ), (forest, JJS), (high, JJ), (sch...",-0.125,0.375
19536,804134698729385984,oh jeff bruh,"[(oh, UH), (jeff, NN), (bruh, NN)]",0.000,0.000
19537,804126501117435904,richard sherman weighs cam newton struggle sea...,"[(richard, NN), (sherman, NN), (weighs, VBZ), ...",0.000,0.000


In [ ]:
f1 = pd.DataFrame()
f1["id"] = data["id"]
f1["senti_score"] = data["senti_score"]
f1["senti_score_absolute"] = data["senti_score_absolute"]

In [ ]:
f1.head()

,id,senti_score,senti_score_absolute
0,858462320779026433,-0.125,0.875
1,858421020331560960,0.750,0.750
2,858368123753435136,0.000,0.750
3,858323428260139008,0.875,0.875
4,858283602626347008,0.000,0.000


In [ ]:
f1.head(10)

,id,senti_score,senti_score_absolute
0,858462320779026433,-0.125,0.875
1,858421020331560960,0.750,0.750
2,858368123753435136,0.000,0.750
3,858323428260139008,0.875,0.875
4,858283602626347008,0.000,0.000
5,858224473597779969,-0.125,0.375
6,858222698367000577,0.125,0.375
7,858191667739987968,0.125,0.125
8,858153572571197440,-0.750,0.750
9,858150052782645249,-1.375,1.375


Add sentiment features in existing csv file 

In [ ]:
f1.to_csv("/content/gdrive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/Features/sentimentPostText",index=False)